# LangChain Expression Language

### What is LCEL?

* A declarative way to build AI pipelines in LangChain.
* Instead of long Python code, you connect components in a chain-like style.

---

### Why LCEL?

* Shorter and cleaner code.
* Easier to read and maintain.
* Works the same for synchronous, asynchronous, batch, and streaming execution.
* Good for debugging and reusing components.

---

### Basic Example

Without LCEL:

```python
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)
result = chain.run("cats")
```

With LCEL:

```python
chain = prompt | ChatOpenAI() | StrOutputParser()
result = chain.invoke({"topic": "cats"})
```
**prompt, ChatOpenAI(), StrOutputParser() are called runnables**

---

## LCEL Operators

* `|` → Pipe operator, connects steps.
* `.invoke()` → Run once with input.
* `.batch()` → Run for multiple inputs.
* `.stream()` → Get outputs step by step (streaming responses).

---

## Key Features

1. Composable – you can plug and play different parts.
2. Parallel and sequential execution supported.
3. Unified API – works the same for sync, async, batch, and streaming.
4. Production-ready – optimized for speed and scalability.

---

## Simple Summary

* LCEL is a short and readable way to connect LangChain components.
* It works like piping data through different steps.
* Use `|` to chain steps.
* The same code works for real-time, async, or batch calls.

---


# ➡️Piping a prompt, model, and an output parser

In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda, chain
import config

In [2]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [3]:
# Chat Template

chat_template = ChatPromptTemplate.from_messages([
    ('human', "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)
])

print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [4]:
# Chat Model

def AskGeminiAI(question):
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=config.gemini_api_key,
        temperature=0.7,
        max_tokens=250,
        model_kwargs={"seed": 42}
    )
    response = llm.invoke(question)
    return response.content

In [5]:
# Output parser

list_output_parser = CommaSeparatedListOutputParser()

In [6]:
# 3 step

chat_template_result = chat_template.invoke({'pet':'dog'})

chat_result = AskGeminiAI(chat_template_result)

list_output_parser.invoke(chat_result)

['Luna', 'Cooper', 'Bella']

In [7]:
# Single line

chain = chat_template | AskGeminiAI | list_output_parser

chain.invoke({'pet':'dog'})

['Buddy', 'Luna', 'Charlie']

# ➡️Batching

In [8]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', 
     "I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?")
])

In [9]:
chain = chat_template | AskGeminiAI

In [10]:
chain.invoke({'pet':'dog', 'breed':'shepherd'})

"Congratulations on adopting your Shepherd! They're intelligent and trainable dogs, but they do require consistent and positive training. Here are some training tips tailored to Shepherds:\n\n**1. Understand Your Shepherd's Breed Characteristics:**\n\n*   **Intelligence:** Shepherds are highly intelligent and eager to please. This makes them quick learners, but also means they can get bored easily. Keep training sessions short, engaging, and varied.\n*   **Herding Instinct:** Many Shepherds retain a strong herding instinct. This can manifest as nipping at heels, chasing cars, or rounding up children. Training should focus on redirecting this instinct into more appropriate behaviors.\n*   **Protectiveness:** They are naturally protective of their family and territory. Early socialization is crucial to prevent over-protectiveness and aggression towards strangers or other animals.\n*   **High Energy:** Shepherds need a lot of exercise and mental stimulation to thrive. A bored Shepherd can

In [11]:
%%time
chain.batch([{'pet':'dog', 'breed':'shepherd'}, 
             {'pet':'dragon', 'breed':'night fury'}])

CPU times: total: 78.1 ms
Wall time: 3.98 s


["Congratulations on adopting a shepherd! They are intelligent, energetic, and loyal dogs, but they do require consistent training and socialization. Here are some training tips specifically geared towards shepherds:\n\n**1. Start Early and Be Consistent:**\n\n*   **Puppyhood is Key:** If you have a shepherd puppy, start training and socialization immediately. Even short, positive training sessions can make a big difference.\n*   **Consistency is Crucial:** Everyone in the household needs to use the same commands and training techniques. This prevents confusion for your dog.\n*   **Establish Yourself as the Leader:** Shepherds are herding dogs, and they need a strong, consistent leader. This doesn't mean being harsh, but it does mean being clear, fair, and confident.\n\n**2. Focus on Basic Obedience:**\n\n*   **Sit, Stay, Come, Down, Leave It:** These are the foundational commands. Master these first. Use positive reinforcement (treats, praise, toys) when your dog performs the desired 

In [12]:
%%time
chain.invoke({'pet':'dog', 'breed':'shepherd'})

CPU times: total: 15.6 ms
Wall time: 2.75 s


'Congratulations on adopting a Shepherd! They are intelligent and eager to please, but also need consistent training and a job to do. Here are some training tips tailored for Shepherds:\n\n**1. Start Early and Be Consistent:**\n\n*   **Begin Immediately:** Even if your dog is an adult, start training right away. Reinforce good behaviors and discourage unwanted ones.\n*   **Consistency is Key:** Everyone in the household needs to use the same commands and training methods. This avoids confusion and helps your dog learn faster.\n*   **Short, Frequent Sessions:** Shepherds have good attention spans, but short, focused training sessions (5-10 minutes, several times a day) are more effective than long, drawn-out ones.\n\n**2. Focus on Basic Obedience:**\n\n*   **Sit, Stay, Down, Come:** These are the foundation of all training. Use positive reinforcement (treats, praise, toys) when your dog performs the command correctly.\n*   **Leash Training:** Shepherds can be prone to pulling on the lea

In [13]:
%%time
chain.invoke({'pet':'dragon', 'breed':'night fury'})

CPU times: total: 62.5 ms
Wall time: 4.18 s


'Congratulations on adopting a Night Fury! They are incredibly intelligent, loyal, and powerful dragons. Training one takes patience, understanding, and a strong bond. Here are some training tips, keeping in mind that Night Furies are sensitive and respond best to positive reinforcement:\n\n**1. Bonding and Trust are Paramount:**\n\n*   **Earn their trust:** Night Furies are naturally wary of humans. Start by offering them food, preferably fish (they love it!), and let them approach you. Avoid sudden movements or loud noises that might scare them.\n*   **Spend quality time:** Just being around your Night Fury, even without actively training, builds trust. Read to them, groom them (they enjoy having their scales scratched around their chin and neck), or simply sit near them.\n*   **Understand their body language:** Pay close attention to their ears, eyes, and posture. A relaxed Night Fury will have relaxed posture, soft eyes, and slightly drooped ears. Tense posture, narrowed eyes, and 

### 1. **CPU time**

* The amount of time the CPU actually spent executing your code.
* Includes user + system CPU usage.
* If your code runs in **parallel** across multiple cores, CPU time can be **greater than wall time** (because it sums across cores).

---

### 2. **Wall time**

* The real-world elapsed time (like looking at a stopwatch).
* Starts when the cell begins execution, stops when it ends.
* Includes waiting, I/O delays, and any time the process was idle.

---

# ➡️Streaming

In [14]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', 
     "I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?")
])

In [15]:
llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=config.gemini_api_key,
        temperature=0.7,
        max_tokens=250,
        model_kwargs={"seed": 42}
    )

In [16]:
chain = chat_template | llm

In [17]:
%%time
chain.invoke({'pet':'dog', 'breed':'shepherd'})

CPU times: total: 93.8 ms
Wall time: 2.67 s


AIMessage(content='Congratulations on adopting a shepherd! They are intelligent and eager to please, but also need consistent training. Here are some training tips, broken down by category:\n\n**1. Understanding Your Shepherd\'s Breed Traits:**\n\n*   **High Energy:** Shepherds were bred to work all day. Be prepared for a dog that needs a lot of physical and mental stimulation.\n*   **Intelligence:** They learn quickly, but can also get bored easily. Keep training sessions short, engaging, and varied.\n*   **Herding Instincts:** They may try to "herd" children, other pets, or even cars. This is a natural instinct, not aggression. Redirect this behavior with appropriate training.\n*   **Protective:** Shepherds are naturally protective of their family and territory. Early socialization is crucial to prevent over-protectiveness or aggression.\n*   **Loyal & Bonded:** They form strong bonds with their owners and thrive on positive interaction.\n\n**2. Basic Obedience Training:**\n\n*   **S

In [18]:
%%time
chain.batch([{'pet':'dog', 'breed':'shepherd'}, 
             {'pet':'dragon', 'breed':'night fury'}])

CPU times: total: 0 ns
Wall time: 3.87 s


[AIMessage(content='Congratulations on adopting a Shepherd! They\'re intelligent and eager to please, but require consistent training and clear leadership. Here are several training tips, broken down into categories:\n\n**1. Establishing Yourself as the Leader (Essential for Shepherds):**\n\n*   **Consistency is Key:** Shepherds thrive on routine and clear rules. Be consistent with commands, boundaries, and expectations. If you say "sit," always mean "sit."\n*   **Positive Reinforcement:** Focus on rewarding good behavior with treats, praise, and toys. Avoid punishment, which can lead to fear and anxiety in Shepherds.\n*   **Clear and Firm Commands:** Use a clear, confident tone of voice when giving commands. Avoid yelling, which can be counterproductive.\n*   **Be Patient:** Training takes time and patience. Don\'t get discouraged by setbacks. Celebrate small victories.\n*   **Mental Stimulation:** Shepherds are working dogs. They need mental challenges to prevent boredom and destruct

In [19]:
response = chain.stream({'pet':'dragon', 'breed':'night fury'})

# next(response) 
# Helps to retrieve the first chunk of output generated by the model.

for i in response:
    print(i.content, end = '')

Congratulations on adopting a Night Fury! They're incredibly intelligent and loyal creatures, but require a specific approach to training. Here's a breakdown of training tips, focusing on building trust and understanding:

**I. Building a Strong Foundation (Trust & Respect):**

*   **Patience is Key:** Night Furies are sensitive and intelligent. Rushing the process will only damage your bond. Start slow and celebrate small victories.
*   **Positive Reinforcement:** Forget punishment! Night Furies respond best to rewards. Use fish (their favorite!), praise, and gentle scratches under the chin.
*   **Observation is Crucial:**  Pay close attention to your dragon's body language.  Learn to recognize signs of fear, stress, excitement, and contentment.  A relaxed Night Fury is a receptive one.
*   **Establish Yourself as a Leader (Not a Master):**  Leadership is earned, not demanded.  Be consistent, fair, and predictable.  Provide a safe and secure environment.  Your dragon needs to trust yo

### 1. `chain.stream(...)`

* Runs the LCEL chain in **streaming mode**.
* Instead of waiting for the **full response**, it returns an **iterator (generator)**.
* Each `yield` gives you a partial output (token or chunk).

---

### 2. `next(response)`

* The first call gets the very first chunk from the model’s output.
* This is often needed to “start” the stream before looping.
* Sometimes developers use it to **inspect metadata** or headers before reading the actual content.

---

### 3. `for i in response:`

* Iterates through the rest of the streamed chunks.
* Each `i` is usually a `ChatGenerationChunk` or a token-like object.

---

### 4. `print(i.content, end='')`

* Prints the text as it streams in, without adding new lines.
* This makes the output look like the model is typing word by word.

---

### Example Flow

If the model response is:

```
"Night Furies are rare dragons known for their speed."
```

Streaming loop prints:

```
Night Furies are rare dragons known for their speed.
```

…but in reality, it arrives piece by piece:

* `Night`
* ` Furies`
* ` are`
* ` rare`
* …and so on.

---


# ➡️The Runnable and RunnableSequence classes

In [20]:
chain = chat_template | llm

In [21]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [22]:
type(chat_template)

langchain_core.prompts.chat.ChatPromptTemplate

### Runnable in LangChain

* A **Runnable** is just something that can be “run” with input and gives output.
* Examples:

  * A prompt template (`PromptTemplate`)
  * An LLM (`ChatOpenAI`, `Gemini`)
  * An output parser

All of them share the same interface: `.invoke(input)`

---

### RunnableSequence

* A **`RunnableSequence`** is a chain of Runnables connected together.
* Think of it as a **pipeline**:

  * Output of one step → becomes input to the next step.
* It is created when you use the **pipe operator (`|`)**.

---

# ➡️Piping chains and the RunnablePassthrough class

In [23]:
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

### What is `RunnablePassthrough`?

* It’s a special kind of **Runnable** in LangChain.
* **Whatever input you give it, it returns the same thing as output.**
* No changes, no processing — just passes the data through.

---

### Why is it useful?

* Sometimes you want to **keep the original input** available in a chain.
* Example: You build a chain where one branch transforms the input, but another branch just needs the raw input.
* `RunnablePassthrough` helps in those cases.

---


In [3]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [4]:
string_parser = StrOutputParser()

In [5]:
llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=config.gemini_api_key,
        temperature=0.7,
        max_tokens=250,
        model_kwargs={"seed": 42}
    )

In [6]:
# A chain that lists the most essential tools for a given prodession
chain_tools = chat_template_tools | llm | string_parser

# A chain that suggests effective strategies for mastering these tools
chain_strategy = chat_template_strategy | llm | string_parser

In [28]:
print(chain_tools.invoke({'job title':'data scientist'}))

1.  Python
2.  SQL
3.  R
4.  Machine Learning Frameworks (e.g., scikit-learn, TensorFlow, PyTorch)
5.  Data Visualization Tools (e.g., Matplotlib, Seaborn, Tableau)


In [29]:
print(chain_strategy.invoke({'tools':'''
1.  Python
2.  SQL
3.  R
4.  Cloud Computing (e.g., AWS, Azure, GCP)
5.  Machine Learning Frameworks (e.g., TensorFlow, PyTorch, scikit-learn)
'''}))

## Strategy for Learning and Mastering Python, SQL, R, Cloud Computing, and Machine Learning Frameworks

This strategy focuses on a structured, hands-on approach, emphasizing practical application and continuous learning. It's broken down into phases and incorporates feedback loops for optimal progress.

**Phase 1: Foundation Building (Weeks 1-4)**

**Goal:** Establish a solid foundational understanding of each tool.  Focus on the core concepts and syntax.

**Tools:**

*   **Python:**
    *   **Resources:**  "Python Crash Course" by Eric Matthes, Codecademy, freeCodeCamp.org.
    *   **Focus:**  Data types, control flow (if/else, loops), functions, basic data structures (lists, dictionaries), file I/O, and fundamental libraries like `os` and `math`.
    *   **Practice:**  Write simple scripts to automate tasks, solve basic coding challenges on platforms like HackerRank or LeetCode (easy difficulty).
*   **SQL:**
    *   **Resources:**  SQLZoo, Mode Analytics SQL Tutorial, freeCodeCamp.

In [7]:
# Combining two chains

chain_tools = chat_template_tools | llm | string_parser | {'tools':RunnablePassthrough()}
# The output of {'tools':RunnablePassthrough()} is dictionary in the format:
# {'tools':'''
# 1.  Python
# 2.  SQL
# 3.  R
# 4.  Cloud Computing (e.g., AWS, Azure, GCP)
# 5.  Machine Learning Frameworks (e.g., TensorFlow, PyTorch, scikit-learn)
# '''}

chain_strategy = chat_template_strategy | llm | string_parser

chain_combined = chain_tools | chain_strategy

In [31]:
print(chain_combined.invoke({'job title':'data scientist'}))

Okay, here's a structured learning strategy for mastering Python, SQL, R, Cloud Computing, and Machine Learning Frameworks, designed to build a strong foundation and practical skills. This strategy emphasizes a hands-on, project-based approach, combining theoretical knowledge with real-world application.

**I. Overall Learning Philosophy:**

*   **Project-Based Learning:**  Focus on building projects that integrate multiple tools.  This is the most effective way to learn and retain knowledge.
*   **Iterative Learning:**  Don't aim for perfection upfront. Start with the basics, build something simple, then iterate and improve as you learn more.
*   **Consistent Practice:**  Dedicate regular time (e.g., 1-2 hours daily) to practice and work on projects. Consistency is key.
*   **Community Engagement:**  Join online forums, attend meetups, and connect with other learners.  Learning from others and sharing your knowledge is invaluable.
*   **Documentation is Your Friend:**  Learn to read a

In [8]:
chain_long = (chat_template_tools | llm | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | llm | string_parser)

# ➡️Graphing Runnables

In [27]:
# pip install grandalf

chain_long.get_graph().print_ascii()
# An ASCII graph is simply a graph (diagram, chart, or plot) drawn using plain text characters (letters, numbers, dashes, pipes, etc.) 
# from the ASCII character set — instead of using images.

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
             *             
             *             
             *             
      +-------------+      
      | Passthrough |      
      +-------------+      
             *             
             *             
             *      

# ➡️RunnableParallel

In [17]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

In [18]:
chain_books = chat_template_books | llm | string_parser

chain_projects = chat_template_projects | llm | string_parser

In [19]:
# Runnable Parallel Objest
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects}) # Key can be anything

In [22]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '*   *Fluent Python* by Luciano Ramalho\n*   *Python Cookbook* by David Beazley and Brian K. Jones\n*   *Effective Python* by Brett Slatkin',
 'projects': '*   **Web Scraper with Data Analysis and Visualization**\n*   **Interactive Command-Line Application for Task Management**\n*   **Simple Game with AI Opponent**'}

In [23]:
chain_parallel.get_graph().print_ascii()

                   +-------------------------------+                     
                   | Parallel<books,projects>Input |                     
                   +-------------------------------+                     
                       ****                  ****                        
                   ****                          ****                    
                 **                                  **                  
  +--------------------+                       +--------------------+    
  | ChatPromptTemplate |                       | ChatPromptTemplate |    
  +--------------------+                       +--------------------+    
             *                                            *              
             *                                            *              
             *                                            *              
+------------------------+                   +------------------------+  
| ChatGoogleGenerativeAI |            

In [24]:
%%time
chain_books.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 1.55 s


'*   *Fluent Python* by Luciano Ramalho\n*   *Python Cookbook* by David Beazley and Brian K. Jones\n*   *Effective Python* by Brett Slatkin'

In [25]:
%%time
chain_projects.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 1.21 s


'1.  **Web Scraper with Data Analysis and Visualization**\n2.  **Interactive Command-Line Application with Database Integration**\n3.  **Simple Game Engine using Pygame or similar library**'

In [26]:
%%time
chain_parallel.invoke({'programming language':'Python'})

CPU times: total: 15.6 ms
Wall time: 3.01 s


{'books': '*   *Fluent Python* by Luciano Ramalho\n*   *Python Cookbook* by David Beazley and Brian K. Jones\n*   *Effective Python* by Brett Slatkin',
 'projects': '1.  Web Scraping and Data Analysis Tool\n2.  Simple Game Engine with Pygame\n3.  REST API with Flask or FastAPI'}

# ➡️Piping a RunnableParallel with other Runnables

In [28]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [30]:
chain_books = chat_template_books | llm | string_parser

chain_projects = chat_template_projects | llm | string_parser

In [31]:
chain_time1 = (RunnableParallel({'books':chain_books, 
                                'projects':chain_projects}) 
              | chat_template_time 
              | llm 
              | string_parser
             )

In [33]:
chain_time2 = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | llm 
              | string_parser
             )

In [36]:
%%time
print(chain_time1.invoke({'programming language':'Python'}))

Okay, let's break down the estimated time commitment for each book and project, considering your intermediate level and aiming for a good understanding rather than just skimming.

**Literature:**

*   **Fluent Python (Luciano Ramalho):** This is a deep dive into Python's more advanced features and idioms.  It's a substantial book.

    *   **Estimated Time:** 80-120 hours.  This assumes you're reading actively, trying out examples, and taking notes.  It's a book to be absorbed, not just read.

*   **Python Cookbook (David Beazley & Brian K. Jones):** This is a recipe-based book, meaning you can jump around to topics of interest.  You don't need to read it cover-to-cover.

    *   **Estimated Time:** 40-80 hours.  This depends on how many recipes you want to explore in detail.  You'll likely be referencing this book throughout your projects.  Focus on sections relevant to your current needs.

*   **Effective Python (Brett Slatkin):** This book is highly practical and filled with concret

In [37]:
%%time
print(chain_time2.invoke({'programming language':'Python'}))

Okay, let's break down the estimated time commitment for reading the books and completing the projects, considering you're at an intermediate programming level.  This is, of course, a *very rough* estimate, as individual learning speeds, existing knowledge, and time availability vary greatly.

**Literature Time Estimates:**

*   **Fluent Python (Luciano Ramalho):** This is a deep dive into Python's core features. It's not a quick read.  It's best absorbed by actively trying out the concepts and examples.
    *   **Reading & Basic Understanding:** 60-80 hours.  This assumes you're dedicating focused time and actively engaging with the material (coding examples, taking notes).  If you're just passively reading, you won't retain as much.
    *   **Applying & Internalizing:**  Another 40-60 hours.  This is where you truly cement the knowledge.  You'll want to revisit sections as needed when working on projects.  This time is interwoven with project work.
    *   **Total (Effective Reading 

In [39]:
chain_time1.get_graph().print_ascii()

                   +-------------------------------+                     
                   | Parallel<books,projects>Input |                     
                   +-------------------------------+                     
                       ****                  ****                        
                   ****                          ****                    
                 **                                  **                  
  +--------------------+                       +--------------------+    
  | ChatPromptTemplate |                       | ChatPromptTemplate |    
  +--------------------+                       +--------------------+    
             *                                            *              
             *                                            *              
             *                                            *              
+------------------------+                   +------------------------+  
| ChatGoogleGenerativeAI |            

# ➡️RunnableLambda

In [41]:
find_sum = lambda x: sum(x)

find_sum([1, 2, 5])

8

In [42]:
find_square = lambda x: x**2

find_square(8)

64

In [45]:
runnable_sum = RunnableLambda(lambda x: sum(x))

runnable_sum.invoke([1, 2, 5])

8

In [46]:
runnable_square = RunnableLambda(lambda x: x**2)

runnable_square.invoke(8)

64

In [47]:
chain = runnable_sum | runnable_square

chain.invoke([1, 2, 5])

64

In [48]:
chain.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 


# ➡️The @chain decorator

In [50]:
def find_sum(x):
    return sum(x)

def find_square(x):
    return x**2

In [51]:
chain1 = RunnableLambda(find_sum) | RunnableLambda(find_square)

chain1.invoke([1, 2, 5])

64

In [53]:
# Chain Decorator - automatically converts the function into runnable lamnbda functions 
@chain
def runnable_sum(x):
    return sum(x)

@chain
def runnable_square(x):
    return x**2

In [52]:
type(runnable_sum), type(runnable_square)

(langchain_core.runnables.base.RunnableLambda,
 langchain_core.runnables.base.RunnableLambda)

In [55]:
chain2 = runnable_sum | runnable_square

chain2.invoke([1, 2, 5])

64